### Import libraries

In [5]:
import pandas as pd
import numpy as np
from functools import reduce
import os
import re
from pyscripts.settings import output_dir, wosis_dir


### Leemos el archivo descargado

In [6]:
df_orgc = pd.read_csv(wosis_dir + "wosis_latest_orgc_Argentina.csv")
df_orgc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gml_id             841 non-null    object 
 1   profile_id         841 non-null    int64  
 2   profile_layer_id   841 non-null    int64  
 3   country_name       841 non-null    object 
 4   upper_depth        841 non-null    int64  
 5   lower_depth        841 non-null    int64  
 6   layer_name         789 non-null    object 
 7   litter             827 non-null    float64
 8   orgc_value         841 non-null    object 
 9   orgc_value_avg     841 non-null    float64
 10  orgc_method        841 non-null    object 
 11  orgc_date          841 non-null    object 
 12  orgc_dataset_id    841 non-null    object 
 13  orgc_profile_code  841 non-null    object 
 14  orgc_licence       841 non-null    object 
dtypes: float64(2), int64(4), object(9)
memory usage: 98.7+ KB


## Parte 1: Obtener los promedios ponderados de un material para los perfiles de un pais

### Calculamos el carbono concentrado de un perfil para los primeros 30cm

la columna orgc_value_avg tiene los valores de carbono para una capa. Solo se deben considerar los primeros 30cm asi que si una capa sobrepasa ese limite se debe tomar un parcial. Para calcular el carbono de esa capa se hace un promedio ponderado de los parciales de cada capa. 

vamos a tomar los primeros tres perfiles que aparecen en el dataset como ejemplo, y solo vamos a tener en cuenta los primeros 30 cm. Esto se puede hacer usando las columnas upper_depth y lower_depth, que nos indican los limites superiores e inferiores de la capa en cuestion. 

Por ejemplo, del perfil 63821, vamos a tomar las primeras dos capas y solo 7cm de la tercer capa.

In [7]:
# Nos quedamos con las columnas relevantes
cols1 = ['profile_id', 'profile_layer_id', 'upper_depth', 'lower_depth', 'orgc_value_avg']
dff = df_orgc[0:9][cols1]
dff

,profile_id,profile_layer_id,upper_depth,lower_depth,orgc_value_avg
0,63821,53190,0,7,9.7
1,63821,53191,7,23,11.9
2,63821,53192,23,45,14.0
3,63821,53193,45,100,6.8
4,63822,53194,0,25,5.0
5,63823,53195,0,17,2.0
6,63823,53196,17,34,2.0
7,63823,53197,34,58,3.0
8,63823,53198,58,100,4.0


### Calculo de promedios ponderados de concentracion de carbono organico

### funcion de agrupamiento

https://pandas.pydata.org/docs/user_guide/groupby.html#

tiene q ser una funcion a nivel de fila. 

toda la informacion que necesito esta en upper y lower, ahi tengo si me pase de 30cm y cuanto mide la layer. Con eso puedo calcular el ponderado en una columna nueva y despues cuando hago el groupby lo puedo dividir por el largo total.

In [8]:
def poderate_avg(profiles, property, limit):
    col = property + '_value_avg'
    if(profiles['upper_depth'] > limit):
        return 0.0 

    if(profiles['lower_depth'] > limit):
        partial_depth = limit - profiles['upper_depth']
        return (partial_depth * profiles[col])/limit
    else:
        return ((profiles['lower_depth'] - profiles['upper_depth']) * profiles[col])/limit


dff['orgc_pond_val'] = dff.apply(poderate_avg, args=('orgc', 30), axis=1)
dff[['profile_id', 'orgc_pond_val']].groupby('profile_id').sum().reset_index()

,profile_id,orgc_pond_val
0,63821,11.876667
1,63822,4.166667
2,63823,2.000000


Para que funcione bien este codigo es importante que no se cambien los nombres de las columnas. Ni de los archivos

Ya obtuvimos los promedios ponderados para cada perfil. El analisis para el resto de las propiedades del perfil (clay, bulk density) es similar. 

Lo que resta ahora es aplicar esta metodologia para todas las propiedades que hagan falta y luego mergear los datasets. Luego de mergear, van a quedar espacios en nulo para aquellos perfiles que no posean al menos una de las propiedades usadas. En cada caso se vera si se puede conseguir la informacion de otro lugar. 

### Traemos el resto de los datasets para hacerles el mismo calculo y luego mergearlos

In [6]:
df_bulk = pd.read_csv( wosis_dir + "wosis_latest_bdfi33_Argentina.csv")
df_clay = pd.read_csv( wosis_dir + "wosis_latest_clay_Argentina.csv")

In [7]:
df_bulk.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gml_id               55 non-null     object 
 1   profile_id           55 non-null     int64  
 2   profile_layer_id     55 non-null     int64  
 3   country_name         55 non-null     object 
 4   upper_depth          55 non-null     int64  
 5   lower_depth          55 non-null     int64  
 6   layer_name           52 non-null     object 
 7   litter               55 non-null     int64  
 8   bdfi33_value         55 non-null     object 
 9   bdfi33_value_avg     55 non-null     float64
 10  bdfi33_method        55 non-null     object 
 11  bdfi33_date          55 non-null     object 
 12  bdfi33_dataset_id    55 non-null     object 
 13  bdfi33_profile_code  55 non-null     object 
 14  bdfi33_licence       55 non-null     object 
dtypes: float64(1), int64(5), object(9)
memory 

In [8]:
df_clay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gml_id             857 non-null    object 
 1   profile_id         857 non-null    int64  
 2   profile_layer_id   857 non-null    int64  
 3   country_name       857 non-null    object 
 4   upper_depth        857 non-null    int64  
 5   lower_depth        857 non-null    int64  
 6   layer_name         820 non-null    object 
 7   litter             857 non-null    int64  
 8   clay_value         857 non-null    object 
 9   clay_value_avg     857 non-null    float64
 10  clay_method        857 non-null    object 
 11  clay_date          857 non-null    object 
 12  clay_dataset_id    857 non-null    object 
 13  clay_profile_code  857 non-null    object 
 14  clay_licence       857 non-null    object 
dtypes: float64(1), int64(5), object(9)
memory usage: 100.6+ KB


Aplicamos la funcion 'ponderate' sobre los tres datasets

In [9]:
def get_profile_summary(df, property):
    prop_pond_col = property + '_pond_val'
    df[prop_pond_col] = df.apply(poderate_avg, args=(property, 30), axis=1)
    return df[['profile_id', prop_pond_col]].groupby('profile_id').sum().reset_index()


luego los unimos en uno solo para obtener la informacion resumida de la concentracion de cada propiedad en cada perfil.

In [10]:
dfs = [df_orgc, df_clay, df_bulk]
properties = ['orgc', 'clay', 'bdfi33']
grouped = []

for df, property in zip(dfs, properties):
    grouped.append(get_profile_summary(df, property))
profille_summary = reduce(lambda l, r: pd.merge(l, r, on='profile_id', how='outer'), grouped)
profille_summary

,profile_id,orgc_pond_val,clay_pond_val,bdfi33_pond_val
0,63821,11.876667,45.533333,NaN
1,63822,4.166667,12.500000,NaN
2,63823,2.000000,16.133333,NaN
3,63824,17.500000,31.666667,NaN
4,63825,16.556667,10.533333,NaN
...,...,...,...,...
244,71870,NaN,6.300000,NaN
245,71919,NaN,3.000000,NaN
246,71950,NaN,8.166667,NaN
247,176590,NaN,19.583333,NaN


In [11]:
profille_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   profile_id       249 non-null    int64  
 1   orgc_pond_val    213 non-null    float64
 2   clay_pond_val    227 non-null    float64
 3   bdfi33_pond_val  12 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 9.7 KB


La mayoria de los valores en bdfi33 estan en nulo porque solo hay 12 perfiles con informacion de bulk density wn wosis. *Por lo que esta informacion necesariamente tiene que obtenerse de otra fuente*

## Parte 2: Obtenemos todos los archivos para todos los paises y repetimos el procedimiento haciendo un resumen por perfil para todos los paises de interes

Primero se deben obtener todos los archivos que se tengan. Asumimos que se encuentran en el directorio wosis_latest dentro de este repositorio y que respetan el siguiente formato de nombre:

*wosis_latest_property_country.format*

donde:
- property: clay, orgc, bdfi33, etc
- country: Argentina, Uruguay, etc (comienzan con mayuscula)
- format: gpkg, csv, etc (shp no funciona aun)

ejemplo:

*wosis_latest_clay_Argentina.csv*

En cuanto a los archivos, podemos asumir que existen o crearlos dentro del script. De todas maneras, para leerlos, se escanea el directorio en busca de los archivos para evitar errores de file not found. No hay nada que procesar si no hay archivos.

### Buscamos los archivos existentes

Para buscar los archivos, tenemos que tener definido un formato. De otra forma traeriamos informacion repetida (repetida?? o cada formato trae algo distinto?? **CONFIRMAR ESTO**)

In [12]:
## we compose a regular expression to help us search for the files we want to process

format = 'csv' # format is fixed
countries = '|'.join(['Argentina', 'Uruguay', 'Chile'])
properties = '|'.join(['clay', 'orgc', 'bdfi33'])

files_re = re.compile(f'wosis_latest_({properties})_({countries}).{format}')
files = []

with os.scandir(wosis_dir) as entries:
    for entry in entries:
        if re.search(files_re, entry.name):
            print(entry.name)
            files.append(entry.name)

wosis_latest_clay_Argentina.csv
wosis_latest_orgc_Uruguay.csv
wosis_latest_bdfi33_Uruguay.csv
wosis_latest_clay_Chile.csv
wosis_latest_clay_Uruguay.csv
wosis_latest_bdfi33_Argentina.csv
wosis_latest_orgc_Argentina.csv
wosis_latest_bdfi33_Chile.csv
wosis_latest_orgc_Chile.csv


Por cada archivo encontrado, se obtiene el dataset y se procesa. Al final se guarda todo en un summary, como en la parte 1

Para incluir al pais en el proceso, hay que modificar la funcion get_profile_summary. Una asumicion importante es que no va a haber un perfil con el mismo ID en otro pais, por lo que no es necesario unir por pais y por perfil cuando se hace el merge. 

In [13]:
# primero vamos a redefinir la funcion de summary para que incluya al pais
def get_profile_summary(df, property):
    prop_pond_col = property + '_pond_val'
    df[prop_pond_col] = df.apply(poderate_avg, args=(property, 30), axis=1)
    return df[['profile_id', 'country_name', prop_pond_col]].groupby(['profile_id', 'country_name']).sum().reset_index()

No hay nada por lo que unir en datasets de distintos paises porque no hay ningun profile que se comparta, sin embargo en el merge hay que unir por pais para que pandas no cree una columna nueva para ese campo. 


Lo que tiene mas sentido entonces, es mergear los datasets de cada pais para obtener los resumenes de cada perfil y despues stackear los datasets para obtener uno grande con todos los paises. 

In [14]:
group = []
countries = {}
for file in files:
    if(format == 'csv'):
        df = pd.read_csv(wosis_dir + file)
        property = file.split("_")[-2]
        country = file.split("_")[-1].split(".")[0]
        if country not in countries.keys():
            countries[country] = []
        countries[country].append(get_profile_summary(df, property))

reduced_countries = {}
for key in countries:
    reduced_countries[key] = reduce(lambda l, r: pd.merge(l, r, on=['profile_id', 'country_name'], how='outer'), countries[key])
        
profille_summary = pd.concat(reduced_countries.values())
profille_summary

,profile_id,country_name,clay_pond_val,bdfi33_pond_val,orgc_pond_val
0,63821,Argentina,45.533333,NaN,11.876667
1,63822,Argentina,12.500000,NaN,4.166667
2,63823,Argentina,16.133333,NaN,2.000000
3,63824,Argentina,31.666667,NaN,17.500000
4,63825,Argentina,10.533333,NaN,16.556667
...,...,...,...,...,...
10786,732831,Chile,NaN,NaN,127.310000
10787,732832,Chile,NaN,NaN,2.000000
10788,732833,Chile,NaN,NaN,1.000000
10789,732834,Chile,NaN,NaN,4.000000


In [15]:
profille_summary[profille_summary['profile_id'] == 68646]

,profile_id,country_name,clay_pond_val,bdfi33_pond_val,orgc_pond_val
7,68646,Uruguay,7.666667,NaN,10.2


In [17]:
profille_summary.to_csv(output_dir + 'profile_summary.csv')